In [3]:
import os
import pandas as pd

df_total =  []
for file_name in os.listdir('./data/product'):
    df_total.append(pd.read_csv(os.path.join('./data/product', file_name)))


df_last = pd.concat(df_total, axis = 0)

In [5]:
df_last.to_csv('./data/final.csv', index = False)

In [5]:
for _, row in df_last.iterrows():
            content = (
                f'Tên sản phẩm: {row["item_name"]} - '
                f'Giá gốc: {row["origin_price"]} - '
                f'Màu sắc và giá tương ứng: {row["color_price"]}\n'
                f'Thông số kĩ thuật: {row["technical_infomation"]}\n\n'
            )
            print(content)

Tên sản phẩm: Xiaomi Redmi Note 14 6GB/128GB - Giá gốc: 4,990,000 ₫ - Màu sắc và giá tương ứng: []
Thông số kĩ thuật: ['Màn hình: [Công nghệ màn hình: Kính Corning® Gorilla® Glass 5\nTần số quét (Hz): 120Hz\nĐộ phân giải: 2400 x 1080\nKích thước màn hình: 6.67"\nĐộ sáng màn hình: 1800 nits\n]', 'Camera sau: [Độ phân giải camera: 108MP\nQuay phim: Camera chính 108MP\n0.64μm, 1.92μm gộp điểm ảnh 9 trong 1\n- f/1.7\nThấu kính 6P\nKích thước cảm biến 1/1.67”\nCamera đo chiều sâu 2MP\n- f/2.4\nCamera cận cảnh 2MP\nTính năng: Google Gemini⁵, Xóa vật thể AI, AI Sky, AI Beautify\n]', 'Camera trước: [Độ phân giải camera: 20MP\n]', 'Hệ điều hành & CPU: [Vi xử lý: MediaTek Helio G99-Ultra\nTốc độ CPU: 2.2GHz\nVi xử lý đồ họa (GPU): Mali-G57 MC2\nHệ điều hành: Xiaomi HyperOS\n]', 'Bộ nhớ & Lưu trữ: [Thẻ nhớ ngoài: Thẻ nhớ mở rộng lên đến 1TB\nRAM: 6GB\nBộ nhớ trong: 128GB\n]', 'Kết nối: [Cảm biến: Cảm biến vân tay tích hợp trong màn hình\nKết Nối NFC: Có\nMạng di động: 2G\n4G\n3G\nSố khe SIM: SIM 